In [6]:
%pip install -qU langchain langchain_community langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install -qU python-dotenv

from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, separators = ["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")
store = InMemoryVectorStore.from_documents(chunks, embeddings)

retriever = store.as_retriever()

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

llm = ChatOpenAI(model = "gpt-4o")

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that can answer questions about the US Constitution. Use the provided context to answer the question. IMPORTANT: If the context does not provide the answer, say 'I don't know' and don't make up the answer."),
#     ("user", "Question: {question}\nContext: {context}")
# ])
prompt = hub.pull("rlm/rag-prompt")

chain = prompt | llm

In [ ]:
query = "What does the constitution say about pardons"

#rag

#retrieval
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

#augmented + generation
response = chain.invoke({
    "question": query,
    "context": docs_content
})

response.content

'The Constitution grants the President the power to grant reprieves and pardons for offenses against the United States, except in cases of impeachment.'